# MATE-SHS - Traitement des données d'un questionnaire

Ce notebook propose de suivre les différentes étapes de l'analyse des données d'une enquête par questionnaire.

**L'enquête est passée par un prestataire, et les résultats sont transmis dans un fichier SPSS .sav**

### Démarche

Les étapes du traitement sont alors :

1. Chargement et recodage
2. Nettoyage et exploration (1. et 2. sont itératifs)
3. Production de statistiques descriptives et de visualisations
4. Traitements statistiques plus avancées autour de certaines questions
5. Production de visualisations finalisées

Le Notebook permet de noter progressivement les idées de l'analyse.

**Pour les besoins pédagogique, la première partie de chargement suit un peu la démarche "progressive", avec un exemple de recodage, puis nous passerons rapidement sur le recodage de chaque variable.**

### Données

Le jeu de données sera celui du Wellcome Trust passé par Gallup à travers le monde pour connaître l'attitude vis-à-vis de la science. Une question générale est alors : retrouve-t-on les mêmes tendances dans tous les pays ?

Les données ainsi que le rapport sont disponibles ici : https://wellcome.org/reports/wellcome-global-monitor-covid-19/2020





## Etape 0. Les bibliothèques

Pour utiliser la puissance de Python, on a besoin de recourir à des outils déjà développés (en Python) regroupés dans des bibliothèques, permettant de manipuler des tableaux, faire des visualisations, etc.

Ces bibliothèques sont regroupées sur https://pypi.org/

Pour les installer, il suffit d'utiliser la commande **pip install NOMDELABIBLIOTHEQUE** soit dans la console, soit dans une cellule.

Chaque bibliothèque a sa philosophie : jeter un coup d'oeil à la documentation est souvent nécessaire.

$\rightarrow$ Installer les bibliothèques : pandas, pyshs, matplotlib

> pip install pandas pyshs matplotlib

Et potentiellement `geopandas` pour les cartes

## Etape 1. Chargement et recodage

Utilisation de la puissance de Pandas : https://pandas.pydata.org/

In [ ]:
# Chargement de bibliothèques
import pandas as pd
import pyshs
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_spss("./wgm-full-wave2-public-file.sav", convert_categoricals=True)
data.shape

In [ ]:
# Ou d'utiliser Pyreadstat pour séparer les méta-données
#import pyreadstat
#data, meta = pyreadstat.read_sav('./wgm-full-wave2-public-file.sav')

In [ ]:
data.head()

C'est un **tableau Pandas** (DataFrame). Il est possible :

- de sélectionner des éléments, de les filtrers
- de faire des opérations similaires à des requêtes
- de faire des traitements plus complexes...

### Commencer à analyser

A partir de là, il est possible d'explorer le jeu de données et de commencer à 1/ identifier les questions qui nous intéressent et 2/ recoder les variables

Dans le cas d'une réflexion qui porte sur le rapport à la science : 

Variables dépendantes d'intérêt:

- 'W1': 'How Much You Know About Science'
- 'W2': 'How Much You Understand the Meaning of Science and Scientists'
- 'W6': 'Trust Science'
- **'W7C': 'Leaders in National Govt Value Opinions/Expertise of Scientists'**
- 'W8': 'Work of Scientists Benefits People in (Country)',
- 'W9': 'Work of Scientists Benefits People Like You in (Country)',
- 'WP21768': 'Agree to Be Vaccinated if Coronavirus Vaccine Was Available at No Cost
- **'MH4A': 'Important for National Govt to Fund Research on Cancer'**
- 'MH4B': 'Important for National Govt to Fund Research on Anxiety/Depression'

Variables indépendantes :
- 'Age': 'Age',
- 'Gender': 'Gender',
- 'Education': 'Education Level',
- 'Household_Income': 'Per Capita Income Quintiles',
- 'Global11Regions': 'Global regions used in report analysis',
- 'wbi': 'Country income level -- World Bank most recent definition',
- 'Subjective_Income': 'Feelings About Household Income',
- 'EMP_2010': 'Employment Status'}
- 'W27': 'Used Social Media in Past 30 Days',
- 'W28': 'How Often Use Social Media',
- 'W29': 'How Often See Information About Health on Social Media',
- 'W30': 'Believe Science or Teachings of Your Religion',
- 'WP21757': 'Extent Life Has Been Affected by Coronavirus Situation
- Différents items sur la confiance : 'W5A': 'Trust People in Neighborhood',
 'W5B': 'Trust the National Government in This Country',
 'W5C': 'Trust Scientists in This Country',
 'W5D': 'Trust Journalists in This Country',
 'W5E': 'Trust Doctors and Nurses in This Country',
 'W5F': 'Trust People Who Work at Charitable Organizations or NGOs in This Country',
 'W5G': 'Trust Traditional Healers in This Country',

Par ailleurs, comme  souvent, c'est une enquête pondérée avec les poids : 'WGT'

### Regarder chaque variable pour la recoder comme on le souhaite

Cas de la place de la science dans la décision politique

Il y a des valeurs nulles et les labels ne sont pas explicites

In [ ]:
data["W7C"].unique()

Passer du type "catégorie" au type générique "object" et changer les valeurs

In [ ]:
# On change de type
data["W7C"] = data["W7C"].astype("object")

# On remplace
data["W7C"] = data["W7C"].replace({"Some":"2-Some", 
                                   "Not at all":"1-Not much or not at al", "Not much":"1-Not much or not at al",
                                   "A lot":"3-A lot", "DK/Refused":"4-DK/Refused"})

# On remplace les valeurs nulles et on change de nom
data["relation_science_politiques"] = data["W7C"].fillna("5-NA")

On regarde la distribution

In [ ]:
data["relation_science_politiques"].value_counts()

In [ ]:
100*data["relation_science_politiques"].value_counts(normalize=True)

L'idée est alors de faire ce traitement pour chacune des variables, en vérifiant graduellement. Et à la fin de rassembler toutes les étapes de codage dans un même bloc comme ci-dessous

In [ ]:
data["connaissance_science"].unique()

In [ ]:
reco = {"Some":"2-Some", 
        "Not at all":"1-Not much or not at al", "Not much":"1-Not much or not at al",
        "A lot":"3-A lot", "DK/Refused":"4-DK/Refused"}
data["connaissance_science"] = data['W1'].astype("object").replace(reco).fillna("5-NA")

In [ ]:
data["connaissance_science"].value_counts()

### Bloc de recodage

In [ ]:
# Recodage des variables utilisées - importance des labels explicits

# Changement de noms
data["poids"] = data["WGT"].astype("object").fillna("NA")
data["pays"] = data["COUNTRYNEW"].astype("object").fillna("NA")
data["age_reco"] = data["age_var1"].astype("object").fillna("NA")
data["genre"] = data["Gender"].astype("object").fillna("NA")
data["education"] = data["Education"].astype("object").fillna("NA")
data["revenus"] = data["Household_Income"].astype("object").fillna("NA")


reco = {"Some":"2-Some", 
        "Not at all":"1-Not much or not at al", "Not much":"1-Not much or not at al",
        "A lot":"3-A lot", "DK/Refused":"4-DK/Refused"}
data["gouvernants_valorisent_scientifiques"] = data["W7C"].replace(reco).astype("object").fillna("4-DK/Refused")
data["confiance_hopital"] = data["W4"].replace(reco).astype("object").fillna("4-DK/Refused")
data["confiance_science"] = data["W6"].replace(reco).astype("object").fillna("4-DK/Refused")
data["confiance_gouvernement"] = data["W5B"].replace(reco).astype("object").fillna("4-DK/Refused")
data["confiance_scientifiques"] = data["W5C"].replace(reco).astype("object").fillna("4-DK/Refused")
data["confiance_journalistes"] = data["W5D"].replace(reco).astype("object").fillna("4-DK/Refused")
data["confiance_medecins"] = data["W5E"].replace(reco).astype("object").fillna("4-DK/Refused")
data["confiance_rebouteux"] = data["W5G"].replace(reco).astype("object").fillna("4-DK/Refused")

$\rightarrow$ Recoder le bénéfice de la science

Si au cours de l'analyse, de nouvelles variables sont utilisées, il est facile de rajouter les variables dans ce bloc pour qu'elles puissent être intégrées

Sauvegarder

## Etape 2. Nettoyage et exploration

Cette étape recoupe la précédente car elle relève aussi largement de l'identification des bonnes variables.

Par exemple, deux questions que nous pouvons nous poser :

1. Est-ce qu'on considère les différentes réponses de question sur la confiance individuellement ou est-ce qu'on construit un indicateur
2. Est-ce que les deux questions sur la confiance dans les scientifiques du pays, et la confiance dans la science en général, est la même chose

In [ ]:
data["W5A"].unique()

In [ ]:
reco = {"Not at all":0,"DK/Refused":0,"Not much":1,"Some":2,"A lot":4}

In [ ]:
# Corrélation entre les indicateurs avec Pandas en enlevant les valeurs absentes
corr = data[['W5A', 'W5B', 'W5C', 'W5D', 'W5E', 'W5F', 'W5G']].replace(reco).corr()
corr

In [ ]:
# Le rendre un peu plus joli avec Seaborn
# https://medium.com/@szabo.bibor/how-to-create-a-seaborn-correlation-heatmap-in-python-834c0686b88e
import seaborn as sns
heatmap = sns.heatmap(corr, vmin=0, vmax=1, annot=True)

On constate la particularité du dernier indicateur ...

Lien entre les deux questions : on fait un tableau croisé

In [ ]:
pd.crosstab(data["confiance_science"],data["confiance_scientifiques"],margins=True)

On peut le représenter sur un tableau croisé

In [ ]:
tab = pd.crosstab(data["confiance_science"],data["confiance_scientifiques"])
tab.plot(kind="bar",cmap="viridis",figsize=(10,5))

Cela ne coincide pas exactement. On va donc créer une typologie pour notre analyse :
- confiance dans la science (W4 = 1,2), mais pas confiance dans les scientifiques (W5D=3,4)
- confiance dans les scientifiques, mais pas dans la science (l'inverse)
- pas confiance dans la science ni les scientifiques
- confiance dans les deux
- si une valeur manque, NA

In [ ]:
data["confiance_scientifiques"].unique()

In [ ]:
def reco(ligne):
    #  Cas où une valeur manque
    if (ligne["confiance_science"]=='4-DK/Refused') | (ligne["confiance_scientifiques"]=='4-DK/Refused'):
        return "5-NA"
    
    # Cas où confiance science mais pas scientifique
    if (ligne["confiance_science"] in ['2-Some', '3-A lot']) & (ligne["confiance_scientifiques"] in ['1-Not much or not at al']):
        return "1-Confiance science mais pas les scientifiques"
    
    # Cas où confiance scientifique mais pas science
    if (ligne["confiance_science"] in ['1-Not much or not at al']) & (ligne["confiance_scientifiques"] in ['2-Some', '3-A lot']):
        return "2-Confiance scientifiques mais pas dans la science"
    
    # Cas où confiance générale
    if (ligne["confiance_science"] in ['2-Some', '3-A lot']) & (ligne["confiance_scientifiques"] in ['2-Some', '3-A lot']):
        return "3-Confiance science"
    
    # Cas où méfiance générale
    if (ligne["confiance_science"] in ['1-Not much or not at al']) & (ligne["confiance_scientifiques"] in ['1-Not much or not at al']):
        return "4-Méfiance science" 
    
# Appliquer le recodage
data["confiance_science_scientifiques"] = data.apply(reco,axis=1)
data["confiance_science_scientifiques"].value_counts()

## Etape 3. Production de données statistiques

Dans le cas ici, on a des données qui sont en plus structurées par pays. Il faut donc penser des traitements par pays (par exemple sur la France) et des comparaisons

On va utiliser une bibliothèque pour faire des statistiques

### Exploration du cas de la France

Construction d'un sous-corpus

In [ ]:
data_fr = data[data["pays"].str.contains("France")]

In [ ]:
data_fr.head()

#### Tri à plats

In [ ]:
pyshs.tri_a_plat(data_fr,"confiance_science_scientifiques","poids")

In [ ]:
tableau = {
    "Confiance Science":pyshs.tri_a_plat(data_fr,"confiance_science","poids"),
    "Confiance Scientifiques":pyshs.tri_a_plat(data_fr,"confiance_scientifiques","poids"),
    "Confiance Hopital":pyshs.tri_a_plat(data_fr,"confiance_hopital","poids"),
    "Confiance Médecins":pyshs.tri_a_plat(data_fr,"confiance_medecins","poids"),
    "Confiance Gouvernement":pyshs.tri_a_plat(data_fr,"confiance_gouvernement","poids"),
}
pd.concat(tableau,axis=1)

### Tableau croisé

In [ ]:
var_ind = {"genre":"Genre","age_reco":"Age",
           "education":"Diplome"}

Rapport Science/politique

In [ ]:
pyshs.tableau_croise_multiple(data_fr,"confiance_hopital",var_ind,"poids")

Pas de grosses variations

#### Petit visualisation

In [ ]:
# Définir en amont les variables
variables = {"confiance_hopital":"Confiance hopital","confiance_science":"Confiance Science",
            "confiance_gouvernement":"Confiance gouvernement","confiance_scientifiques":"confiance scientifiques",
            "confiance_journalistes":"Confiance journalistes","confiance_medecins":"Confiance médecins"}

# Définir une nouvelle figure
fig,ax = plt.subplots(3,2,figsize=(8,10),sharex=True,sharey=True)

# passer le tableau de 2 dimensions à 1
ax = ax.flatten()

# Pour chaque variable à tracer
for i,j in zip(variables,range(0,len(variables))):
    t = pyshs.tri_a_plat(data_fr,i,"poids")["Pourcentage (%)"].drop("Total")
    t.plot(kind='bar',ax=ax[j],color = ["red","orange","green","gray"],alpha=0.5)
    ax[j].set_title(variables[i])
    
fig.suptitle("Confiance des répondants du pays")

#### Par pays, afficher la confiance à l'hopital

In [ ]:
# Construire une fonction dédiée qui traite un tableau pandas
def prop_lot(x):
    t = pyshs.tri_a_plat(x,"confiance_hopital","poids")
    try:
        return t.loc['3-A lot',"Pourcentage (%)"]
    except:
        return None

# L'appliquer à la sortie d'un groupby, trier et afficher
data.groupby("pays").apply(prop_lot).sort_values().plot(kind="barh",figsize=(5,30))

## Analyse statistique plus avancée

Suivant votre spécialité, vous aimeriez faire des traitements. En tant que sociologue, je serai intéressé à connaître les principaux déterminants sur certaines variables d'intérêts, par exemple le fait de penser que les gouvernants valorise l'expertise des scientifiques. Une solution est de faire une régression logistique. Plusieurs solutions pour cela : passer par la bibliothèque statsmodels, ou utiliser le wrapper de pyshs pour obtenir un tableau rapidement

Créer une variable dichotomisée dépendante 1/0

In [ ]:
data["valo_sci_dich"] = data["gouvernants_valorisent_scientifiques"].apply(lambda x : 1 if x=="3-A lot" else 0)

In [ ]:
var_ind = {"genre":"Genre","age_reco":"Age","revenus":"Revenus"}
pyshs.regression_logistique(data,"valo_sci_dich",var_ind,"poids")

## Représentations géographiques

Pour cela on utilise GéoPandas

Il faut un fond de carte, généralement en shapefiles : https://public.opendatasoft.com/explore/dataset/world-administrative-boundaries/export/

In [ ]:
import geopandas as gpd

In [ ]:
carte = gpd.read_file("./world-administrative-boundaries/world-administrative-boundaries.shp")

Vérifions que le nom des pays est écrit de la même manière dans le jeu de données et dans la carte

In [ ]:
noms_cartes = carte["name"].unique()
noms_data = data["COUNTRYNEW"].unique()

for i in noms_data:
    if not i in noms_cartes:
        print(i)

Ce sont les noms qu'il faut relier, donc on va créer un dictionnaire de correspondances et créer une nouvelle colonne avec les noms recodées

In [ ]:
reco = {   "United States":'United States of America',
    "United Kingdom":'U.K. of Great Britain and Northern Ireland',
    "Iran":'Iran (Islamic Republic of)',
    "Tanzania":'United Republic of Tanzania',
     "South Korea":'Republic of Korea',
     'Moldova':'Moldova, Republic of',
     "Russia":'Russian Federation',
     "Bosnia Herzegovina":'Bosnia & Herzegovina',
     "Congo Brazzaville":'Democratic Republic of the Congo',
     "North Macedonia":'The former Yugoslav Republic of Macedonia'
    }

# reste à faire Kosovo et Ivory Coast
data["pays_reco"] = data["pays"].replace(reco) 

Il est possible maintenant de créer des données par pays à partir du sondage

In [ ]:
data_pays = pd.DataFrame(data.groupby("pays_reco").apply(prop_lot),columns=["Confiance hopital"])

Et de les associer à la carte

In [ ]:
carte_data = carte.join(data_pays,on="name")

Pour construire une visualisation (plus de détail par exemple sur  https://geopandas.org/en/stable/docs/user_guide/mapping.html)

In [ ]:
ax = carte_data.plot(column='Confiance hopital',figsize=(15,10),legend=True,
                legend_kwds={'label': "% de répondants qui ont confiance dans l'hopital",
                             'orientation': "horizontal"}, missing_kwds={'color': 'lightgrey'})
ax.set_axis_off()

Aller plus loin avec des cartes interactives : https://python-visualization.github.io/folium/quickstart.html